# CNTK 202: Language Understanding with Recurrent Networks

This tutorial shows how to implement a recurrent network to process text,
for the [Air Travel Information Services](https://catalog.ldc.upenn.edu/LDC95S26) 
(ATIS) task of slot tagging (tag individual words to their respective classes, 
where the classes are provided as labels in the training data set).
We will start with a straight-forward (linear) embedding of the words followed by a recurrent LSTM.
This will then be extended to include neighboring words and run bidirectionally.
Lastly, we will turn this system into an intent classifier.  

The techniques you will practice are:
* model description by composing layer blocks, a convenient way to compose 
  networks/models without requiring the need to write formulas,
* creating your own layer block
* variables with different sequence lengths in the same network
* training the network

We assume that you are familiar with basics of deep learning, and these specific concepts:
* recurrent networks ([Wikipedia page](https://en.wikipedia.org/wiki/Recurrent_neural_network))
* text embedding ([Wikipedia page](https://en.wikipedia.org/wiki/Word_embedding))

### Prerequisites

We assume that you have already [installed CNTK](https://docs.microsoft.com/en-us/cognitive-toolkit/Setup-CNTK-on-your-machine).
This tutorial requires CNTK V2. We strongly recommend to run this tutorial on a machine with
a capable CUDA-compatible GPU. Deep learning without GPUs is not fun.

#### Downloading the data

In this tutorial we are going to use a (lightly preprocessed) version of the ATIS dataset. You can download the data automatically by running the cells below or by executing the manual instructions.

#### Fallback manual instructions
Download the ATIS [training](https://github.com/Microsoft/CNTK/blob/v2.0/Tutorials/SLUHandsOn/atis.train.ctf) 
and [test](https://github.com/Microsoft/CNTK/blob/v2.0/Tutorials/SLUHandsOn/atis.test.ctf) 
files and put them at the same folder as this notebook. If you want to see how the model is 
predicting on new sentences you will also need the vocabulary files for 
[queries](https://github.com/Microsoft/CNTK/blob/v2.0/Examples/LanguageUnderstanding/ATIS/BrainScript/query.wl) and
[slots](https://github.com/Microsoft/CNTK/blob/v2.0/Examples/LanguageUnderstanding/ATIS/BrainScript/slots.wl)

In [1]:
from __future__ import print_function # Use a function definition from future version (say 3.x from 2.7 interpreter)
import requests
import os

def download(url, filename):
    """ utility function to download a file """
    response = requests.get(url, stream=True)
    with open(filename, "wb") as handle:
        for data in response.iter_content():
            handle.write(data)

locations = ['Tutorials/SLUHandsOn', 'Examples/LanguageUnderstanding/ATIS/BrainScript']

data = {
  'train': { 'file': 'atis.train.ctf', 'location': 0 },
  'test': { 'file': 'atis.test.ctf', 'location': 0 },
  'query': { 'file': 'query.wl', 'location': 1 },
  'slots': { 'file': 'slots.wl', 'location': 1 }
}

for item in data.values():
    location = locations[item['location']]
    path = os.path.join('..', location, item['file'])
    if os.path.exists(path):
        print("Reusing locally cached:", item['file'])
        # Update path
        item['file'] = path
    elif os.path.exists(item['file']):
        print("Reusing locally cached:", item['file'])
    else:
        print("Starting download:", item['file'])
        url = "https://github.com/Microsoft/CNTK/blob/v2.0/%s/%s?raw=true"%(location, item['file'])
        download(url, item['file'])
        print("Download completed")


Starting download: atis.train.ctf
Download completed
Starting download: slots.wl
Download completed
Starting download: query.wl
Download completed
Starting download: atis.test.ctf
Download completed


#### Importing CNTK and other useful libraries

CNTK's Python module contains several submodules like `io`, `learner`, and `layers`. We also use NumPy in some cases since the results returned by CNTK work like NumPy arrays.

In [2]:
import math
import numpy as np

import cntk as C

In the block below, we check if we are running this notebook in the CNTK internal test machines by looking for environment variables defined there. We then select the right target device (GPU vs CPU) to test this notebook. In other cases, we use CNTK's default policy to use the best available device (GPU, if available, else CPU).

In [3]:
# Select the right target device when this notebook is being tested:
if 'TEST_DEVICE' in os.environ:
    if os.environ['TEST_DEVICE'] == 'cpu':
        C.device.try_set_default_device(C.device.cpu())
    else:
        C.device.try_set_default_device(C.device.gpu(0))

## Task and Model Structure

The task we want to approach in this tutorial is slot tagging.
We use the [ATIS corpus](https://catalog.ldc.upenn.edu/LDC95S26).
ATIS contains human-computer queries from the domain of Air Travel Information Services,
and our task will be to annotate (tag) each word of a query whether it belongs to a
specific item of information (slot), and which one.

The data in your working folder has already been converted into the "CNTK Text Format."
Let us look at an example from the test-set file `atis.test.ctf`:

    19  |S0 178:1 |# BOS      |S1 14:1 |# flight  |S2 128:1 |# O
    19  |S0 770:1 |# show                         |S2 128:1 |# O
    19  |S0 429:1 |# flights                      |S2 128:1 |# O
    19  |S0 444:1 |# from                         |S2 128:1 |# O
    19  |S0 272:1 |# burbank                      |S2 48:1  |# B-fromloc.city_name
    19  |S0 851:1 |# to                           |S2 128:1 |# O
    19  |S0 789:1 |# st.                          |S2 78:1  |# B-toloc.city_name
    19  |S0 564:1 |# louis                        |S2 125:1 |# I-toloc.city_name
    19  |S0 654:1 |# on                           |S2 128:1 |# O
    19  |S0 601:1 |# monday                       |S2 26:1  |# B-depart_date.day_name
    19  |S0 179:1 |# EOS                          |S2 128:1 |# O

This file has 7 columns:

* a sequence id (19). There are 11 entries with this sequence id. This means that sequence 19 consists
of 11 tokens;
* column `S0`, which contains numeric word indices; the input data is encoded in one-hot vectors.  There are 943 words in the vocabulary, so each word is a 943 element vector of all 0 with a 1 at a vector index chosen to represent that word.  For example the word "from" is represented with a 1 at index 444 and zero everywhere else in the vector. The word "monday" is represented with a 1 at index 601 and zero everywhere else in the vector.
* a comment column denoted by `#`, to allow a human reader to know what the numeric word index stands for;
Comment columns are ignored by the system. `BOS` and `EOS` are special words
to denote beginning and end of sentence, respectively;
* column `S1` is an intent label, which we will only use in the last part of the tutorial;
* another comment column that shows the human-readable label of the numeric intent index;
* column `S2` is the slot label, represented as a numeric index; and
* another comment column that shows the human-readable label of the numeric label index.

The task of the neural network is to look at the query (column `S0`) and predict the
slot label (column `S2`).
As you can see, each word in the input gets assigned either an empty label `O`
or a slot label that begins with `B-` for the first word, and with `I-` for any
additional consecutive word that belongs to the same slot.

The model we will use is a recurrent model consisting of an embedding layer,
a recurrent LSTM cell, and a dense layer to compute the posterior probabilities:


    slot label   "O"        "O"        "O"        "O"  "B-fromloc.city_name"
                  ^          ^          ^          ^          ^
                  |          |          |          |          |
              +-------+  +-------+  +-------+  +-------+  +-------+
              | Dense |  | Dense |  | Dense |  | Dense |  | Dense |  ...
              +-------+  +-------+  +-------+  +-------+  +-------+
                  ^          ^          ^          ^          ^
                  |          |          |          |          |
              +------+   +------+   +------+   +------+   +------+   
         0 -->| LSTM |-->| LSTM |-->| LSTM |-->| LSTM |-->| LSTM |-->...
              +------+   +------+   +------+   +------+   +------+   
                  ^          ^          ^          ^          ^
                  |          |          |          |          |
              +-------+  +-------+  +-------+  +-------+  +-------+
              | Embed |  | Embed |  | Embed |  | Embed |  | Embed |  ...
              +-------+  +-------+  +-------+  +-------+  +-------+
                  ^          ^          ^          ^          ^
                  |          |          |          |          |
    w      ------>+--------->+--------->+--------->+--------->+------... 
                 BOS      "show"    "flights"    "from"   "burbank"

Or, as a CNTK network description. Please have a quick look and match it with the description above:
(descriptions of these functions can be found at: [the layers reference](http://cntk.ai/pythondocs/layerref.html))


In [4]:
# number of words in vocab, slot labels, and intent labels
vocab_size = 943 ; num_labels = 129 ; num_intents = 26    

# model dimensions
input_dim  = vocab_size
label_dim  = num_labels
emb_dim    = 150
hidden_dim = 300

# Create the containers for input feature (x) and the label (y)
x = C.sequence.input_variable(vocab_size)
y = C.sequence.input_variable(num_labels)

def create_model():
    with C.layers.default_options(initial_state=0.1):
        return C.layers.Sequential([
            C.layers.Embedding(emb_dim, name='embed'),
            C.layers.Recurrence(C.layers.LSTM(hidden_dim), go_backwards=False),
            C.layers.Dense(num_labels, name='classify')
        ])

Now we are ready to create a model and inspect it. 

The model attributes are fully accessible from Python. The first layer named `embed` is an Embedding layer. Here we use the CNTK default, which is linear embedding. It is a simple matrix with dimension (input word encoding x output projected dimension). You can access its parameter `E` (where the embeddings are stored) like any other attribute of a Python object. Its shape contains a `-1` which indicates that this parameter (with input dimension) is not fully specified yet, while the output dimension is set to `emb_dim` ( = 150 in this tutorial). 

Additionally we also inspect the value of the bias vector in the `Dense` layer named `classify`. The `Dense` layer is a fundamental compositional unit of a Multi-Layer Perceptron (as introduced in CNTK 103C tutorial). The `Dense` layer has both `weight` and `bias` parameters, one each per `Dense` layer. Bias terms are by default initialized to 0 (but there is a way to change that if you need). As you create the model, one should name the layer component and then access the parameters as shown here. 

**Suggested task**: What should be the expected dimension of the `weight` matrix from the layer named `classify`? Try printing the weight matrix of the `classify` layer? Does it match with your expected size?

In [5]:
# peek
z = create_model()
print(z.embed.E.shape)
print(z.classify.b.value)

(-1, 150)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In our case we have input as one-hot encoded vector of length 943 and the output dimension `emb_dim` is set to 150. In the code below we pass the input variable `x` to our model `z`. This binds the model with input data of known shape. In this case, the input shape will be the size of the input vocabulary. With this modification, the parameter returned by the embed layer is completely specified (943, 150). **Note**: You can initialize the Embedding matrix with pre-computed vectors using [Word2Vec](https://en.wikipedia.org/wiki/Word2vec) or [GloVe](https://en.wikipedia.org/wiki/GloVe_(machine_learning)).

In [6]:
# Pass an input and check the dimension
z = create_model()
print(z(x).embed.E.shape)

(943, 150)


## CNTK Configuration

To train and test a model in CNTK, we need to create a model and specify how to read data and perform training and testing. 

In order to train we need to specify:

* how to read the data 
* the model function, its inputs, and outputs
* hyper-parameters for the learner such as the learning rate

[comment]: <> (For testing ...)

### A Brief Look at Data and Data Reading

We already looked at the data.
But how do you generate this format?
For reading text, this tutorial uses the `CNTKTextFormatReader`. It expects the input data to be
in a specific format, as described [here](https://docs.microsoft.com/en-us/cognitive-toolkit/Brainscript-CNTKTextFormat-Reader).

For this tutorial, we created the corpora by two steps:
* convert the raw data into a plain text file that contains of TAB-separated columns of space-separated text. For example:

  ```
  BOS show flights from burbank to st. louis on monday EOS (TAB) flight (TAB) O O O O B-fromloc.city_name O B-toloc.city_name I-toloc.city_name O B-depart_date.day_name O
  ```

  This is meant to be compatible with the output of the `paste` command.
* convert it to CNTK Text Format (CTF) with the following command:

  ```
  python [CNTK root]/Scripts/txt2ctf.py --map query.wl intent.wl slots.wl --annotated True --input atis.test.txt --output atis.test.ctf
  ```
  where the three `.wl` files give the vocabulary as plain text files, one word per line.

In these CTF files, our columns are labeled `S0`, `S1`, and `S2`.
These are connected to the actual network inputs by the corresponding lines in the reader definition:

In [7]:
def create_reader(path, is_training):
    return C.io.MinibatchSource(C.io.CTFDeserializer(path, C.io.StreamDefs(
         query         = C.io.StreamDef(field='S0', shape=vocab_size,  is_sparse=True),
         intent_unused = C.io.StreamDef(field='S1', shape=num_intents, is_sparse=True),  
         slot_labels   = C.io.StreamDef(field='S2', shape=num_labels,  is_sparse=True)
     )), randomize=is_training, max_sweeps = C.io.INFINITELY_REPEAT if is_training else 1)

In [8]:
# peek
reader = create_reader(data['train']['file'], is_training=True)
reader.streams.keys()

dict_keys(['intent_unused', 'slot_labels', 'query'])

### Trainer

We also must define the training criterion (loss function), and also an error metric to track. In most tutorials, we know the input dimensions and the corresponding labels. We directly create the loss and the error functions. In this tutorial we will do the same. However, we take a brief detour and learn about placeholders. This concept would be useful for Task 3. 

**Learning note**: Introduction to `placeholder`: Remember that the code we have been writing is not actually executing any heavy computation it is just specifying the function we want to compute on data during training/testing. And in the same way that it is convenient to have names for arguments when you write a regular function in a programming language, it is convenient to have placeholders that refer to arguments (or local computations that need to be reused). Eventually, some other code will replace these placeholders with other known quantities in the same way that in a programming language the function will be called with concrete values bound to its arguments. 

Specifically, the input variables you have created above `x = C.sequence.input_variable(vocab_size)` holds data pre-defined by `vocab_size`. In the case where such instantiations are challenging or not possible, using `placeholder` is a logical choice. Having the `placeholder` only allows you to defer the specification of the argument at a later time when you may have the data.

Here is an example below that illustrates the use of `placeholder`.

In [9]:
def create_criterion_function(model):
    labels = C.placeholder(name='labels')
    ce   = C.cross_entropy_with_softmax(model, labels)
    errs = C.classification_error      (model, labels)
    return C.combine ([ce, errs]) # (features, labels) -> (loss, metric)

criterion = create_criterion_function(create_model())
criterion.replace_placeholders({criterion.placeholders[0]: C.sequence.input_variable(num_labels)})

Composite(Combine): Input('Input2300', [#, *], [129]), Placeholder('labels', [???], [???]) -> Output('Block2270_Output_0', [#, *], [???]), Output('Block2290_Output_0', [#, *], [???])

While the cell above works well when one has input parameters defined at network creation, it compromises readability. Hence we prefer creating functions as shown below

In [10]:
def create_criterion_function_preferred(model, labels):
    ce   = C.cross_entropy_with_softmax(model, labels)
    errs = C.classification_error      (model, labels)
    return ce, errs # (model, labels) -> (loss, error metric)

In [11]:
def train(reader, model_func, max_epochs=10):
    
    # Instantiate the model function; x is the input (feature) variable 
    model = model_func(x)
    
    # Instantiate the loss and error function
    loss, label_error = create_criterion_function_preferred(model, y)

    # training config
    epoch_size = 18000        # 18000 samples is half the dataset size 
    minibatch_size = 70
    
    # LR schedule over epochs 
    # In CNTK, an epoch is how often we get out of the minibatch loop to
    # do other stuff (e.g. checkpointing, adjust learning rate, etc.)
    # (we don't run this many epochs, but if we did, these are good values)
    lr_per_sample = [0.003]*4+[0.0015]*24+[0.0003]
    lr_per_minibatch = [lr * minibatch_size for lr in lr_per_sample]
    lr_schedule = C.learning_rate_schedule(lr_per_minibatch, C.UnitType.minibatch, epoch_size)
    
    # Momentum schedule
    momentum_as_time_constant = C.momentum_as_time_constant_schedule(700)
    
    # We use a the Adam optimizer which is known to work well on this dataset
    # Feel free to try other optimizers from 
    # https://www.cntk.ai/pythondocs/cntk.learner.html#module-cntk.learner
    learner = C.adam(parameters=model.parameters,
                     lr=lr_schedule,
                     momentum=momentum_as_time_constant,
                     gradient_clipping_threshold_per_sample=15, 
                     gradient_clipping_with_truncation=True)

    # Setup the progress updater
    progress_printer = C.logging.ProgressPrinter(tag='Training', num_epochs=max_epochs)
    
    # Uncomment below for more detailed logging
    #progress_printer = ProgressPrinter(freq=100, first=10, tag='Training', num_epochs=max_epochs) 

    # Instantiate the trainer
    trainer = C.Trainer(model, (loss, label_error), learner, progress_printer)

    # process minibatches and perform model training
    C.logging.log_number_of_parameters(model)

    t = 0
    for epoch in range(max_epochs):         # loop over epochs
        epoch_end = (epoch+1) * epoch_size
        while t < epoch_end:                # loop over minibatches on the epoch
            data = reader.next_minibatch(minibatch_size, input_map={  # fetch minibatch
                x: reader.streams.query,
                y: reader.streams.slot_labels
            })
            trainer.train_minibatch(data)               # update model with it
            t += data[y].num_samples                    # samples so far
        trainer.summarize_training_progress()

### Running it

You can find the complete recipe below.

In [12]:
def do_train():
    global z
    z = create_model()
    reader = create_reader(data['train']['file'], is_training=True)
    train(reader, z)
do_train()

Training 721479 parameters in 6 parameter tensors.
Learning rate per minibatch: 0.21
Finished Epoch[1 of 10]: [Training] loss = 0.692392 * 18010, metric = 14.14% * 18010 12.116s (1486.5 samples/s);
Finished Epoch[2 of 10]: [Training] loss = 0.196757 * 18051, metric = 4.43% * 18051 12.790s (1411.3 samples/s);
Finished Epoch[3 of 10]: [Training] loss = 0.127802 * 17941, metric = 2.88% * 17941 12.581s (1426.0 samples/s);
Finished Epoch[4 of 10]: [Training] loss = 0.089395 * 18059, metric = 2.14% * 18059 9.858s (1831.9 samples/s);
Learning rate per minibatch: 0.105
Finished Epoch[5 of 10]: [Training] loss = 0.057005 * 17957, metric = 1.33% * 17957 11.274s (1592.8 samples/s);
Finished Epoch[6 of 10]: [Training] loss = 0.051911 * 18021, metric = 1.22% * 18021 11.044s (1631.7 samples/s);
Finished Epoch[7 of 10]: [Training] loss = 0.047058 * 17980, metric = 1.17% * 17980 10.159s (1769.9 samples/s);
Finished Epoch[8 of 10]: [Training] loss = 0.040754 * 18025, metric = 1.07% * 18025 10.097s (178

This shows how learning proceeds over epochs (passes through the data).
For example, after four epochs, the loss, which is the cross-entropy criterion, has reached 0.11 as measured on the ~18000 samples of this epoch,
and that the error rate is 2.6% on those same 18000 training samples.

The epoch size is the number of samples--counted as *word tokens*, not sentences--to
process between model checkpoints.

Once the training has completed (a little less than 2 minutes on a Titan-X or a Surface Book),
you will see an output like this
```
Finished Epoch [10]: [Training] loss = 0.033263 * 18039, metric = 0.9% * 18039
```
which is the loss (cross entropy) and the metric (classification error) averaged over the final epoch.

On a CPU-only machine, it can be 4 or more times slower. You can try setting
```python
emb_dim    = 50 
hidden_dim = 100
```
to reduce the time it takes to run on a CPU, but the model will not fit as well as when the 
hidden and embedding dimension are larger. 

### Evaluating the model

Like the train() function, we also define a function to measure accuracy on a test set by computing the error over multiple minibatches of test data. For evaluating on a small sample read from a file, you can set a minibatch size reflecting the sample size and run the test_minibatch on that instance of data. To see how to evaluate a single sequence, we provide an instance later in the tutorial. 

In [13]:
def evaluate(reader, model_func):
    
    # Instantiate the model function; x is the input (feature) variable 
    model = model_func(x)
    
    # Create the loss and error functions
    loss, label_error = create_criterion_function_preferred(model, y)

    # process minibatches and perform evaluation
    progress_printer = C.logging.ProgressPrinter(tag='Evaluation', num_epochs=0)

    while True:
        minibatch_size = 500
        data = reader.next_minibatch(minibatch_size, input_map={  # fetch minibatch
            x: reader.streams.query,
            y: reader.streams.slot_labels
        })
        if not data:                                 # until we hit the end
            break

        evaluator = C.eval.Evaluator(loss, progress_printer)
        evaluator.test_minibatch(data)
     
    evaluator.summarize_test_progress()

Now we can measure the model accuracy by going through all the examples in the test set and using the ``test_minibatch`` method of the trainer created inside the evaluate function defined above. At the moment (when this tutorial was written) the Trainer constructor requires a learner (even if it is only used to perform ``test_minibatch``) so we have to specify a dummy learner. In the future it will be allowed to construct a Trainer without specifying a learner as long as the trainer only calls ``test_minibatch``

In [14]:
def do_test():
    reader = create_reader(data['test']['file'], is_training=False)
    evaluate(reader, z)
do_test()
z.classify.b.value

Finished Evaluation [1]: Minibatch[1-23]: metric = 0.34% * 10984;


array([-3.44756097e-02, -9.48742926e-02, -4.88143601e-02, -5.89377098e-02,
       -1.66121200e-02, -4.74706963e-02, -4.71220650e-02, -9.88641307e-02,
       -2.75263712e-02, -5.82716130e-02, -3.17683741e-02, -3.77656259e-02,
       -5.17959744e-02, -5.13010696e-02, -8.52241889e-02, -7.09928051e-02,
       -1.22144140e-01, -4.68127318e-02,  4.03327495e-02, -1.32786557e-01,
       -7.63492659e-02, -5.49318455e-02,  5.47909038e-03, -7.66143017e-03,
       -4.85182293e-02,  1.35481404e-02, -3.55821103e-02, -3.82119953e-03,
       -1.73545275e-02, -3.66954207e-02, -3.91245857e-02, -3.24573815e-02,
       -7.47283399e-02, -1.02308849e-02, -5.27024157e-02,  9.11082923e-02,
        6.01251870e-02, -2.08287835e-02, -1.44987218e-02, -4.56239879e-02,
       -1.41239911e-01, -6.53960556e-02,  1.86713673e-02, -4.64309864e-02,
        1.97561365e-02, -8.04510191e-02,  3.41979191e-02,  2.72030365e-02,
        2.97282673e-02, -4.36839908e-02, -6.07293919e-02, -8.45976621e-02,
        2.59138849e-02, -

The following block of code illustrates how to evaluate a single sequence. Additionally we show how one can pass in the information using NumPy arrays.

In [15]:
# load dictionaries
query_wl = [line.rstrip('\n') for line in open(data['query']['file'])]
slots_wl = [line.rstrip('\n') for line in open(data['slots']['file'])]
query_dict = {query_wl[i]:i for i in range(len(query_wl))}
slots_dict = {slots_wl[i]:i for i in range(len(slots_wl))}

# let's run a sequence through
seq = 'BOS flights from new york to seattle EOS'
w = [query_dict[w] for w in seq.split()] # convert to word indices
print(w)
onehot = np.zeros([len(w),len(query_dict)], np.float32)
for t in range(len(w)):
    onehot[t,w[t]] = 1

#x = C.sequence.input_variable(vocab_size)
pred = z(x).eval({x:[onehot]})[0]
print(pred.shape)
best = np.argmax(pred,axis=1)
print(best)
list(zip(seq.split(),[slots_wl[s] for s in best]))

[178, 429, 444, 619, 937, 851, 752, 179]
(8, 129)
[128 128 128  48 110 128  78 128]


[('BOS', 'O'),
 ('flights', 'O'),
 ('from', 'O'),
 ('new', 'B-fromloc.city_name'),
 ('york', 'I-fromloc.city_name'),
 ('to', 'O'),
 ('seattle', 'B-toloc.city_name'),
 ('EOS', 'O')]

## Modifying the Model

In the following, you will be given tasks to practice modifying CNTK configurations.
The solutions are given at the end of this document... but please try without!

### A Word About [`Sequential()`](https://www.cntk.ai/pythondocs/layerref.html#sequential)

Before jumping to the tasks, let's have a look again at the model we just ran.
The model is described in what we call *function-composition style*.
```python
        Sequential([
            Embedding(emb_dim),
            Recurrence(LSTM(hidden_dim), go_backwards=False),
            Dense(num_labels)
        ])
```
You may be familiar with the "sequential" notation from other neural-network toolkits.
If not, [`Sequential()`](https://www.cntk.ai/pythondocs/layerref.html#sequential) is a powerful operation that,
in a nutshell, allows to compactly express a very common situation in neural networks
where an input is processed by propagating it through a progression of layers.
`Sequential()` takes an list of functions as its argument,
and returns a *new* function that invokes these functions in order,
each time passing the output of one to the next.
For example,
```python
	FGH = Sequential ([F,G,H])
    y = FGH (x)
```
means the same as
```
    y = H(G(F(x))) 
```
This is known as ["function composition"](https://en.wikipedia.org/wiki/Function_composition),
and is especially convenient for expressing neural networks, which often have this form:

         +-------+   +-------+   +-------+
    x -->|   F   |-->|   G   |-->|   H   |--> y
         +-------+   +-------+   +-------+

Coming back to our model at hand, the `Sequential` expression simply
says that our model has this form:

         +-----------+   +----------------+   +------------+
    x -->| Embedding |-->| Recurrent LSTM |-->| DenseLayer |--> y
         +-----------+   +----------------+   +------------+

### Task 1: Add Batch Normalization

We now want to add new layers to the model, specifically batch normalization.

Batch normalization is a popular technique for speeding up convergence.
It is often used for image-processing setups, for example our other [hands-on lab on image
recognition](./Hands-On-Labs-Image-Recognition).
But could it work for recurrent models, too?

> Note: training with Batch Normalization is currently only supported on GPU.

So your task will be to insert batch-normalization layers before and after the recurrent LSTM layer.
If you have completed the [hands-on labs on image processing](https://github.com/Microsoft/CNTK/blob/v2.0/Tutorials/CNTK_201B_CIFAR-10_ImageHandsOn.ipynb),
you may remember that the [batch-normalization layer](https://www.cntk.ai/pythondocs/layerref.html#batchnormalization-layernormalization-stabilizer) has this form:
```
    BatchNormalization()
```
So please go ahead and modify the configuration and see what happens.

If everything went right, you will notice improved convergence speed (`loss` and `metric`)
compared to the previous configuration.

In [18]:
# Your task: Add batch normalization
def create_model():
    with C.layers.default_options(initial_state=0.1):
        return C.layers.Sequential([
            C.layers.Embedding(emb_dim),
            C.layers.Recurrence(C.layers.LSTM(hidden_dim), go_backwards=False),
            C.layers.Dense(num_labels)
        ])

# Enable these when done:
z = create_model()
do_train()
do_test()

Training 721479 parameters in 6 parameter tensors.
Learning rate per minibatch: 0.21
Finished Epoch[1 of 10]: [Training] loss = 0.683917 * 18010, metric = 13.86% * 18010 9.510s (1893.8 samples/s);
Finished Epoch[2 of 10]: [Training] loss = 0.195469 * 18051, metric = 4.44% * 18051 9.504s (1899.3 samples/s);
Finished Epoch[3 of 10]: [Training] loss = 0.128745 * 17941, metric = 2.78% * 17941 10.218s (1755.8 samples/s);
Finished Epoch[4 of 10]: [Training] loss = 0.093684 * 18059, metric = 2.19% * 18059 10.869s (1661.5 samples/s);
Learning rate per minibatch: 0.105
Finished Epoch[5 of 10]: [Training] loss = 0.057643 * 17957, metric = 1.35% * 17957 10.864s (1652.9 samples/s);
Finished Epoch[6 of 10]: [Training] loss = 0.054528 * 18021, metric = 1.25% * 18021 11.624s (1550.3 samples/s);
Finished Epoch[7 of 10]: [Training] loss = 0.048111 * 17980, metric = 1.20% * 17980 9.738s (1846.4 samples/s);
Finished Epoch[8 of 10]: [Training] loss = 0.041791 * 18025, metric = 1.04% * 18025 10.396s (1733.

### Task 2: Add a Lookahead 

Our recurrent model suffers from a structural deficit:
Since the recurrence runs from left to right, the decision for a slot label
has no information about upcoming words. The model is a bit lopsided.
Your task will be to modify the model such that
the input to the recurrence consists not only of the current word, but also of the next one
(lookahead).

Your solution should be in function-composition style.
Hence, you will need to write a Python function that does the following:

* takes no input arguments
* creates a placeholder (sequence) variable
* computes the "next value" in this sequence using the `sequence.future_value()` operation and
* concatenates the current and the next value into a vector of twice the embedding dimension using `splice()`

and then insert this function into `Sequential()`'s list right after the embedding layer.

In [19]:
# Your task: Add lookahead
def create_model():
    with C.layers.default_options(initial_state=0.1):
        return C.layers.Sequential([
            C.layers.Embedding(emb_dim),
            C.layers.Recurrence(C.layers.LSTM(hidden_dim), go_backwards=False),
            C.layers.Dense(num_labels)
        ])
    
# Enable these when done:
z = create_model()
do_train()
do_test()

Training 721479 parameters in 6 parameter tensors.
Learning rate per minibatch: 0.21
Finished Epoch[1 of 10]: [Training] loss = 0.803032 * 18010, metric = 15.88% * 18010 24.261s (742.3 samples/s);
Finished Epoch[2 of 10]: [Training] loss = 0.236665 * 18051, metric = 5.31% * 18051 26.285s (686.7 samples/s);
Finished Epoch[3 of 10]: [Training] loss = 0.158289 * 17941, metric = 3.70% * 17941 38.955s (460.6 samples/s);
Finished Epoch[4 of 10]: [Training] loss = 0.110174 * 18059, metric = 2.64% * 18059 30.323s (595.6 samples/s);
Learning rate per minibatch: 0.105
Finished Epoch[5 of 10]: [Training] loss = 0.071051 * 17957, metric = 1.74% * 17957 31.652s (567.3 samples/s);
Finished Epoch[6 of 10]: [Training] loss = 0.065072 * 18021, metric = 1.51% * 18021 29.990s (600.9 samples/s);
Finished Epoch[7 of 10]: [Training] loss = 0.053745 * 17980, metric = 1.33% * 17980 30.551s (588.5 samples/s);
Finished Epoch[8 of 10]: [Training] loss = 0.049872 * 18025, metric = 1.20% * 18025 28.652s (629.1 sam

### Task 3: Bidirectional Recurrent Model

Aha, knowledge of future words help. So instead of a one-word lookahead,
why not look ahead until all the way to the end of the sentence, through a backward recurrence?
Let us create a bidirectional model!

Your task is to implement a new layer that
performs both a forward and a backward recursion over the data, and
concatenates the output vectors.

Note, however, that this differs from the previous task in that
the bidirectional layer contains learnable model parameters.
In function-composition style,
the pattern to implement a layer with model parameters is to write a *factory function*
that creates a *function object*.

A function object, also known as [*functor*](https://en.wikipedia.org/wiki/Function_object), is an object that is both a function and an object.
Which means nothing else that it contains data yet still can be invoked as if it was a function.

For example, `Dense(outDim)` is a factory function that returns a function object that contains
a weight matrix `W`, a bias `b`, and another function to compute 
`input @ W + b.` (This is using 
[Python 3.5 notation for matrix multiplication](https://docs.python.org/3/whatsnew/3.5.html#whatsnew-pep-465).
In Numpy syntax it is `input.dot(W) + b`).
E.g. saying `Dense(1024)` will create this function object, which can then be used
like any other function, also immediately: `Dense(1024)(x)`. 

Let's look at an example for further clarity: Let us implement a new layer that combines
a linear layer with a subsequent batch normalization. 
To allow function composition, the layer needs to be realized as a factory function,
which could look like this:

```python
def DenseLayerWithBN(dim):
    F = Dense(dim)
    G = BatchNormalization()
    x = placeholder()
    apply_x = G(F(x))
    return apply_x
```

Invoking this factory function will create `F`, `G`, `x`, and `apply_x`. In this example, `F` and `G` are function objects themselves, and `apply_x` is the function to be applied to the data.
Thus, e.g. calling `DenseLayerWithBN(1024)` will
create an object containing a linear-layer function object called `F`, a batch-normalization function object `G`,
and `apply_x` which is the function that implements the actual operation of this layer
using `F` and `G`. It will then return `apply_x`. To the outside, `apply_x` looks and behaves
like a function. Under the hood, however, `apply_x` retains access to its specific instances of `F` and `G`.

Now back to our task at hand. You will now need to create a factory function,
very much like the example above.
You shall create a factory function
that creates two recurrent layer instances (one forward, one backward), and then defines an `apply_x` function
which applies both layer instances to the same `x` and concatenate the two results.

Alright, give it a try! To know how to realize a backward recursion in CNTK,
please take a hint from how the forward recursion is done.
Please also do the following:
* remove the one-word lookahead you added in the previous task, which we aim to replace; and
* make sure each LSTM is using `hidden_dim//2` outputs to keep the total number of model parameters limited.

In [20]:
# Your task: Add bidirectional recurrence
def create_model():
    with C.layers.default_options(initial_state=0.1):  
        return C.layers.Sequential([
            C.layers.Embedding(emb_dim),
            C.layers.Recurrence(C.layers.LSTM(hidden_dim), go_backwards=False),
            C.layers.Dense(num_labels)
        ])

# Enable these when done:
#do_train()
#do_test()

Works like a charm! This model achieves 0.32%, better than the lookahead model above.
The bidirectional model has 40% less parameters than the lookahead one. However, if you go back and look closely
you may find that the lookahead one trained about 30% faster.
This is because the lookahead model has both less horizontal dependencies (one instead of two
recurrences) and larger matrix products, and can thus achieve higher parallelism.

### Solution 1: Adding Batch Normalization

In [21]:
def create_model():
    with C.layers.default_options(initial_state=0.1):
        return C.layers.Sequential([
            C.layers.Embedding(emb_dim),
            #C.layers.BatchNormalization(),
            C.layers.Recurrence(C.layers.LSTM(hidden_dim), go_backwards=False),
            #C.layers.BatchNormalization(),
            C.layers.Dense(num_labels)
        ])

do_train()
do_test()

Training 721479 parameters in 6 parameter tensors.
Learning rate per minibatch: 0.21
Finished Epoch[1 of 10]: [Training] loss = 0.788779 * 18010, metric = 15.44% * 18010 24.858s (724.5 samples/s);
Finished Epoch[2 of 10]: [Training] loss = 0.224621 * 18051, metric = 5.14% * 18051 23.324s (773.9 samples/s);
Finished Epoch[3 of 10]: [Training] loss = 0.146788 * 17941, metric = 3.34% * 17941 24.133s (743.4 samples/s);
Finished Epoch[4 of 10]: [Training] loss = 0.100341 * 18059, metric = 2.34% * 18059 23.369s (772.8 samples/s);
Learning rate per minibatch: 0.105
Finished Epoch[5 of 10]: [Training] loss = 0.065097 * 17957, metric = 1.58% * 17957 25.488s (704.5 samples/s);
Finished Epoch[6 of 10]: [Training] loss = 0.059250 * 18021, metric = 1.45% * 18021 26.087s (690.8 samples/s);
Finished Epoch[7 of 10]: [Training] loss = 0.052901 * 17980, metric = 1.35% * 17980 22.496s (799.3 samples/s);
Finished Epoch[8 of 10]: [Training] loss = 0.044244 * 18025, metric = 1.12% * 18025 26.256s (686.5 sam

### Solution 2: Add a Lookahead

In [22]:
def OneWordLookahead():
    x = C.placeholder()
    apply_x = C.splice(x, C.sequence.future_value(x))
    return apply_x

def create_model():
    with C.layers.default_options(initial_state=0.1):
        return C.layers.Sequential([
            C.layers.Embedding(emb_dim),
            OneWordLookahead(),
            C.layers.Recurrence(C.layers.LSTM(hidden_dim), go_backwards=False),
            C.layers.Dense(num_labels)        
        ])

do_train()
do_test()

Training 901479 parameters in 6 parameter tensors.
Learning rate per minibatch: 0.21
Finished Epoch[1 of 10]: [Training] loss = 0.620051 * 18010, metric = 12.55% * 18010 26.932s (668.7 samples/s);
Finished Epoch[2 of 10]: [Training] loss = 0.169874 * 18051, metric = 3.91% * 18051 27.183s (664.1 samples/s);
Finished Epoch[3 of 10]: [Training] loss = 0.109912 * 17941, metric = 2.37% * 17941 25.793s (695.6 samples/s);
Finished Epoch[4 of 10]: [Training] loss = 0.077911 * 18059, metric = 1.77% * 18059 26.788s (674.1 samples/s);
Learning rate per minibatch: 0.105
Finished Epoch[5 of 10]: [Training] loss = 0.043471 * 17957, metric = 1.07% * 17957 25.232s (711.7 samples/s);
Finished Epoch[6 of 10]: [Training] loss = 0.037294 * 18021, metric = 0.83% * 18021 27.007s (667.3 samples/s);
Finished Epoch[7 of 10]: [Training] loss = 0.036537 * 17980, metric = 0.86% * 17980 25.336s (709.7 samples/s);
Finished Epoch[8 of 10]: [Training] loss = 0.025901 * 18025, metric = 0.67% * 18025 27.829s (647.7 sam

### Solution 3: Bidirectional Recurrent Model

In [23]:
def BiRecurrence(fwd, bwd):
    F = C.layers.Recurrence(fwd)
    G = C.layers.Recurrence(bwd, go_backwards=True)
    x = C.placeholder()
    apply_x = C.splice(F(x), G(x))
    return apply_x 

def create_model():
    with C.layers.default_options(initial_state=0.1):
        return C.layers.Sequential([
            C.layers.Embedding(emb_dim),
            BiRecurrence(C.layers.LSTM(hidden_dim//2), 
                                  C.layers.LSTM(hidden_dim//2)),
            C.layers.Dense(num_labels)
        ])

do_train()
do_test()

Training 541479 parameters in 9 parameter tensors.
Learning rate per minibatch: 0.21
Finished Epoch[1 of 10]: [Training] loss = 0.703717 * 18010, metric = 13.45% * 18010 22.506s (800.2 samples/s);
Finished Epoch[2 of 10]: [Training] loss = 0.175063 * 18051, metric = 3.99% * 18051 24.502s (736.7 samples/s);
Finished Epoch[3 of 10]: [Training] loss = 0.101848 * 17941, metric = 2.21% * 17941 23.234s (772.2 samples/s);
Finished Epoch[4 of 10]: [Training] loss = 0.064707 * 18059, metric = 1.54% * 18059 23.991s (752.7 samples/s);
Learning rate per minibatch: 0.105
Finished Epoch[5 of 10]: [Training] loss = 0.038879 * 17957, metric = 0.92% * 17957 25.608s (701.2 samples/s);
Finished Epoch[6 of 10]: [Training] loss = 0.037028 * 18021, metric = 0.86% * 18021 24.184s (745.2 samples/s);
Finished Epoch[7 of 10]: [Training] loss = 0.033304 * 17980, metric = 0.79% * 17980 25.136s (715.3 samples/s);
Finished Epoch[8 of 10]: [Training] loss = 0.026227 * 18025, metric = 0.65% * 18025 24.425s (738.0 sam